<a href="https://colab.research.google.com/github/jurajhunak/FrequencyWords/blob/master/Pacienti_obce_v3_10_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Novinky

## v 3.9

https://colab.research.google.com/drive/1pzCfFINtVYPA3NoawgcCtrwRvIpomY_z?usp=sharing v tomto scripte boli doplnene chybajuce ID_HP_ZP vo formate "uzsX" tak, ako mi to Peto povedal a mam napisane rucne v poznamkach zo stretka. V casti, kde concatujem uzs_jzs a hp namiesto jednotlivych uzs_jzs filov vlozim upraveny uzs_jzs_allhic.csv

## v 3.10

upravene MS attribution (enngine) podla poslednych Petovych rad - filtracia bola zla, mam hladat co najnizsi subor aj co najnizsi "prior" (co je vraj povodne nejake cislo riadku alebo tak)

# Imports


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
import json

import seaborn as sns

# Datasets

## Hospitalizations all

OSN - General\04_Siet nemocnic\02_Vyhodnotenie_siete\OSN_MedicalServiceGrouping\Kroky_zaloha\uzs_ms_list.csv

In [ ]:
filepath_tab1 = "/content/drive/MyDrive/Colab Notebooks/Jobs/IZA/BBSK vekova struktura/dataset.txt"
hospitalizations = pd.read_csv(filepath_tab1, delimiter="|", low_memory=False)

In [ ]:
hospitalizations.head()

,id_hosp_nem,vek,HMOTNOST,UPV,typ_zs,vykon_lab,vykon,kod_ms,ms_name,drg,icd,icd_vedlajsia,icd_any,joiner,subor,prior,tazkosti,sig_op,vek_skupina,kod_nem,ID_HP_ZP
0,10079_22001847_P3852701,82,0,0,DRG,12j0011_20220708_P,12j0011,26-28,Základné výkony diagnostické a terapeutické vý...,.*,.*,.*,.*,vykon,17,22433,0,0,age19,P3852701,10079_22001847
1,10079_22001847_P3852701,82,0,0,DRG,14j01x_20220708_B,14j01x,26-13,Komplexné výkony na trachey a bronchoch I.,.*,.*,.*,.*,vykon,17,22503,0,0,age19,P3852701,10079_22001847
2,10079_22001847_P3852701,82,0,0,DRG,1j202_20220707_Z,1j202,26-28,Základné výkony diagnostické a terapeutické vý...,.*,.*,.*,.*,vykon,17,22563,0,0,age19,P3852701,10079_22001847
3,10079_22001792_P3852701,75,0,0,DRG,1j202_20220630_Z,1j202,26-28,Základné výkony diagnostické a terapeutické vý...,.*,.*,.*,.*,vykon,17,22563,0,0,age19,P3852701,10079_22001792
4,10079_22001792_P3852701,75,0,0,DRG,14j01x_20220704_B,14j01x,26-13,Komplexné výkony na trachey a bronchoch I.,.*,.*,.*,.*,vykon,17,22503,0,0,age19,P3852701,10079_22001792


In [ ]:
hospitalizations.dtypes

id_hosp_nem      object
vek               int64
HMOTNOST          int64
UPV               int64
typ_zs           object
vykon_lab        object
vykon            object
kod_ms           object
ms_name          object
drg              object
icd              object
icd_vedlajsia    object
icd_any          object
joiner           object
subor             int64
prior             int64
tazkosti          int64
sig_op            int64
vek_skupina      object
kod_nem          object
ID_HP_ZP         object
dtype: object

In [ ]:
hospitalizations.shape

(3644380, 21)

kvoli tomu, ze neboli v ramci stlpca rovnake datatypes, mi neslo neskor spajat. Preto musim stlpec, co potrebujem, zmenit na string

In [ ]:
hospitalizations['ID_HP_ZP'] = hospitalizations['ID_HP_ZP'].astype(str)

In [ ]:
#TODO najpocetnejsie ID_HP_ZP ako vyzeraju

## Files with HP and JZS of all health insurance companies

update od verzie 3.9.: https://colab.research.google.com/drive/1pzCfFINtVYPA3NoawgcCtrwRvIpomY_z?usp=sharing v tomto scripte boli doplnene chybajuce ID_HP_ZP vo formate "uzsX" tak, ako mi to Peto povedal a mam napisane rucne v poznamkach zo stretka. V casti, kde concatujem uzs_jzs a hp namiesto jednotlivych uzs_jzs filov vlozim upraveny uzs_jzs_allhic.csv

In [ ]:
uzs_jzs_allhic = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Jobs/IZA/Pacienti_obce/data/uzs_jzs_allhic.csv')

<ipython-input-9-623d1c5c4ed1>:1: DtypeWarning: Columns (2,5,11,13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  uzs_jzs_allhic = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Jobs/IZA/Pacienti_obce/data/uzs_jzs_allhic.csv')


In [ ]:
uzs_jzs_allhic

,KOD_ZP,BIC_POI,ROK_MESIAC,PZS_12,ID_HP_ZP,NOVORODENEC,DATUM_OD,DATUM_DO,POHYB_POI,DGN_PRIJ,DGN_PREP,KOD_VYKON_JZS,KOD_VYKON_OPER,VYKON_TYP,HOSP_TYP,PZS_ODOSIELATEL,DATUM_INDIKACIE,STAROST_TYP
0,24,3920291.0,2022-03,P38811005101,56771_22089850,NaN,NaN,2022-03-04,I005,F104,F104,NaN,NaN,N,A,N77927005201,NaN,UH
1,24,4039170.0,2022-07,P89851069101,31064_22004566,NaN,2022-07-16,2022-07-22,I001,I2514,I2514,NaN,NaN,N,D,N33067278101,NaN,UH
2,24,3201459.0,2022-09,N80847051101,30909_22003283,1NO,NaN,2022-09-01,I008,Z380,Z380,NaN,NaN,N,A,NaN,NaN,DRG
3,24,7744470.0,2022-01,P76995229101,60666_22000229,NaN,2022-01-22,2022-01-27,I009,C541,C541,NaN,NaN,O,A,P76995025103,NaN,UH
4,24,3267233.0,2022-02,P85363005101,59946_22000645,NaN,NaN,2022-02-10,I020,F078,F078,NaN,NaN,N,A,NaN,NaN,UH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227517,27,NaN,2022-06,N42231009101,uzs1227521,NaN,2022-06-13,2022-06-14,I009,O800,O990,NaN,NaN,N,A,NaN,NaN,UH
1227518,27,NaN,2022-06,P79469009101,uzs1227522,NaN,NaN,NaN,Z009,O800,O420,NaN,NaN,N,A,NaN,NaN,UH
1227519,27,NaN,2022-06,P79469007101,NaN,1NO,NaN,NaN,S008,P011,P011,NaN,NaN,N,A,P79469051101,NaN,UH
1227520,27,NaN,2022-07,P79469009101,NaN,NaN,NaN,NaN,I009,O800,O800,NaN,NaN,O,A,NaN,NaN,UH


zatial nechavam zvysok po starom aj pri nacitani filov, neskor pri upratovani kodu by sa to malo prepisat

In [ ]:
uzs_jzs_24 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Jobs/IZA/Pacienti_obce/data/2022_24_01_UZS_JZS.csv', delimiter='|', low_memory=False)
hp_24 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Jobs/IZA/Pacienti_obce/data/2022_24_02_HP.csv', delimiter='|', low_memory=False)
uzs_jzs_25 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Jobs/IZA/Pacienti_obce/data/2022_25_01_UZS_JZS.csv', delimiter='|', low_memory=False)
hp_25 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Jobs/IZA/Pacienti_obce/data/2022_25_02_HP.csv', delimiter='|', low_memory=False)
uzs_jzs_27 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Jobs/IZA/Pacienti_obce/data/2022_27_01_UZS_JZS.csv', delimiter='|', low_memory=False)
hp_27 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Jobs/IZA/Pacienti_obce/data/2022_27_02_HP.csv', delimiter='|', low_memory=False)

In [ ]:
# changing "ID_HP_ZP" into one data structure
uzs_jzs_24['ID_HP_ZP'] = uzs_jzs_24['ID_HP_ZP'].astype(str)

# Convert "ID_HP_ZP" to float in the second dataframe
hp_24['ID_HP_ZP'] = hp_24['ID_HP_ZP'].astype(str)

# Repeat the same process for the other dataframes
uzs_jzs_25['ID_HP_ZP'] = uzs_jzs_25['ID_HP_ZP'].astype(str)
hp_25['ID_HP_ZP'] = hp_25['ID_HP_ZP'].astype(str)
uzs_jzs_27['ID_HP_ZP'] = uzs_jzs_27['ID_HP_ZP'].astype(str)
hp_27['ID_HP_ZP'] = hp_27['ID_HP_ZP'].astype(str)

### checking their dtypes and structure

In [ ]:
uzs_jzs_25.dtypes

KOD_ZP               int64
BIC_POI            float64
ROK_MESIAC           int64
PZS_12              object
ID_HP_ZP            object
NOVORODENEC         object
DATUM_OD            object
DATUM_DO            object
POHYB_POI           object
DGN_PRIJ            object
DGN_PREP            object
KOD_VYKON_JZS       object
KOD_VYKON_OPER      object
VYKON_TYP           object
HOSP_TYP            object
PZS_ODOSIELATEL     object
DATUM_INDIKACIE     object
STAROST_TYP         object
dtype: object

In [ ]:
uzs_jzs_24.dtypes


KOD_ZP              int64
BIC_POI             int64
ROK_MESIAC         object
PZS_12             object
ID_HP_ZP           object
NOVORODENEC        object
DATUM_OD           object
DATUM_DO           object
POHYB_POI          object
DGN_PRIJ           object
DGN_PREP           object
KOD_VYKON_JZS      object
KOD_VYKON_OPER     object
VYKON_TYP          object
HOSP_TYP           object
PZS_ODOSIELATEL    object
DATUM_INDIKACIE    object
STAROST_TYP        object
dtype: object

In [ ]:
uzs_jzs_27.dtypes

KOD_ZP               int64
BIC_POI            float64
ROK_MESIAC          object
PZS_12              object
ID_HP_ZP            object
NOVORODENEC         object
DATUM_OD            object
DATUM_DO            object
POHYB_POI           object
DGN_PRIJ            object
DGN_PREP            object
KOD_VYKON_JZS       object
KOD_VYKON_OPER      object
VYKON_TYP           object
HOSP_TYP            object
PZS_ODOSIELATEL     object
DATUM_INDIKACIE     object
STAROST_TYP         object
dtype: object

In [ ]:
hp_25.dtypes

KOD_ZP              int64
ID_HP_PZS          object
ID_HP_ZP           object
BIC_POI           float64
PZS_6              object
DATUM_OD           object
DATUM_DO           object
HP_DLZKA          float64
PRIEPUSTKA_DNI    float64
VEK_DNI             int64
VEK_ROKY            int64
HMOTNOST            int64
UPV_DLZKA           int64
DATUM_NAROD        object
PRIJ_DRUH         float64
PRIJ_DOVOD        float64
PREP_DOVOD        float64
HDG                object
HDG_LOK            object
DRG_SKUP           object
DRG_ERV           float64
ZLUCENE_HP         object
UHRADA_HP         float64
UHRADA_PP         float64
UHRADA_IMZS       float64
UHRADA_EMZS       float64
STAROST_TYP        object
KONTRAKT           object
dtype: object

In [ ]:
uzs_jzs_24.dtypes

KOD_ZP              int64
BIC_POI             int64
ROK_MESIAC         object
PZS_12             object
ID_HP_ZP           object
NOVORODENEC        object
DATUM_OD           object
DATUM_DO           object
POHYB_POI          object
DGN_PRIJ           object
DGN_PREP           object
KOD_VYKON_JZS      object
KOD_VYKON_OPER     object
VYKON_TYP          object
HOSP_TYP           object
PZS_ODOSIELATEL    object
DATUM_INDIKACIE    object
STAROST_TYP        object
dtype: object

In [ ]:
hp_24.dtypes

KOD_ZP              int64
ID_HP_PZS          object
ID_HP_ZP           object
BIC_POI             int64
PZS_6              object
DATUM_OD           object
DATUM_DO           object
HP_DLZKA            int64
PRIEPUSTKA_DNI    float64
VEK_DNI             int64
VEK_ROKY            int64
HMOTNOST            int64
UPV_DLZKA           int64
DATUM_NAROD        object
PRIJ_DRUH           int64
PRIJ_DOVOD          int64
PREP_DOVOD          int64
HDG                object
HDG_LOK            object
DRG_SKUP           object
DRG_ERV           float64
ZLUCENE_HP         object
UHRADA_HP         float64
UHRADA_PP         float64
UHRADA_IMZS         int64
UHRADA_EMZS       float64
STAROST_TYP        object
KONTRAKT           object
dtype: object

## Insurees

In [ ]:
insurees24 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Jobs/IZA/Pacienti_obce/data/2022_24_09_POISTENCI.csv", delimiter='|', low_memory=False)

In [ ]:
insurees25 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Jobs/IZA/Pacienti_obce/data/2022_25_09_POISTENCI.csv", delimiter='|', encoding='ISO-8859-1', low_memory=False)
# bol UnicodeDecodeError, preto ten zvlastny encoding


In [ ]:
insurees27 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Jobs/IZA/Pacienti_obce/data/2022_27_09_POISTENCI.csv", delimiter='|', low_memory=False )

### changing datatypes

spravim z nich najskor integer, aby mali vsetky tvar bez desatinneho miesta. Neskor to bude treba spravit aj s BIC_POI vo finalnom datasete "unique_merged_hospitalizations"

In [ ]:
pomocny_dataset = insurees25[pd.isna(insurees25['KOD_PRECHOD'])]

In [ ]:
pomocny_dataset.head()

,KOD_ZP,BIC_POI,DATUM_NAROD,POHLAVIE,DATUM_POI_OD,DATUM_POI_DO,KONIEC_POI_DOVOD,KOD_PRECHOD,PSC_PRECHOD,KOD_TRVALY,PSC_TRVALY
0,25,8.545430e+09,1975-09-21,M,2005-07-01,NaN,NaN,NaN,NaN,506877,91943
1,25,3.362830e+09,1968-10-01,M,2005-07-01,NaN,NaN,NaN,NaN,508039,90067
2,25,5.682830e+09,1988-10-16,M,2005-07-01,NaN,NaN,NaN,NaN,507938,90028
3,25,6.245440e+09,1945-07-13,Z,2005-07-01,NaN,NaN,NaN,NaN,507865,90083
4,25,9.161440e+09,1939-11-21,Z,2005-07-01,NaN,NaN,NaN,NaN,508098,90042


In [ ]:
pomocny_dataset.shape

(2842387, 11)

In [ ]:
pomocny_dataset['KOD_PRECHOD'] = pomocny_dataset['KOD_PRECHOD'].astype(str)

<ipython-input-25-534ce01ec6a9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pomocny_dataset['KOD_PRECHOD'] = pomocny_dataset['KOD_PRECHOD'].astype(str)


In [ ]:
def convert_to_string(value):
    if pd.isna(value):
        return 'NaN'
    elif isinstance(value, (int, float)):
        return str(int(value))
    else:
        return str(value)

In [ ]:
insurees24['BIC_POI'] = insurees24['BIC_POI'].apply(convert_to_string)
insurees24['KOD_PRECHOD'] = insurees24['KOD_PRECHOD'].apply(convert_to_string)
insurees24['KOD_TRVALY'] = insurees24['KOD_TRVALY'].apply(convert_to_string)

In [ ]:
insurees25['BIC_POI'] = insurees25['BIC_POI'].apply(convert_to_string)
insurees25['KOD_PRECHOD'] = insurees25['KOD_PRECHOD'].apply(convert_to_string)
insurees25['KOD_TRVALY'] = insurees25['KOD_TRVALY'].apply(convert_to_string)

In [ ]:
insurees27['BIC_POI'] = insurees27['BIC_POI'].apply(convert_to_string)
insurees27['KOD_PRECHOD'] = insurees27['KOD_PRECHOD'].apply(convert_to_string)
insurees27['KOD_TRVALY'] = insurees27['KOD_TRVALY'].apply(convert_to_string)

### checking dtypes

In [ ]:
insurees24.dtypes

KOD_ZP                int64
BIC_POI              object
DATUM_NAROD          object
POHLAVIE             object
DATUM_POI_OD         object
DATUM_POI_DO         object
KONIEC_POI_DOVOD    float64
KOD_PRECHOD          object
PSC_PRECHOD         float64
KOD_TRVALY           object
PSC_TRVALY          float64
dtype: object

In [ ]:
insurees24.head()

,KOD_ZP,BIC_POI,DATUM_NAROD,POHLAVIE,DATUM_POI_OD,DATUM_POI_DO,KONIEC_POI_DOVOD,KOD_PRECHOD,PSC_PRECHOD,KOD_TRVALY,PSC_TRVALY
0,24,2372429,1970-12-17,M,1995-06-01,NaN,NaN,NaN,NaN,599816,4012.0
1,24,2372430,1964-04-25,M,1995-06-01,NaN,NaN,522261,4411.0,522261,4411.0
2,24,2372439,1954-12-06,Z,1995-06-01,NaN,NaN,NaN,NaN,521299,4414.0
3,24,2372440,1960-10-25,M,1995-06-01,NaN,NaN,NaN,NaN,522261,4411.0
4,24,2372447,1966-01-29,Z,2023-01-01,NaN,NaN,NaN,NaN,582514,4431.0


In [ ]:
insurees25.shape

(2967758, 11)

In [ ]:
insurees25.dtypes

KOD_ZP               int64
BIC_POI             object
DATUM_NAROD         object
POHLAVIE            object
DATUM_POI_OD        object
DATUM_POI_DO        object
KONIEC_POI_DOVOD    object
KOD_PRECHOD         object
PSC_PRECHOD         object
KOD_TRVALY          object
PSC_TRVALY          object
dtype: object

In [ ]:
insurees25.head()

,KOD_ZP,BIC_POI,DATUM_NAROD,POHLAVIE,DATUM_POI_OD,DATUM_POI_DO,KONIEC_POI_DOVOD,KOD_PRECHOD,PSC_PRECHOD,KOD_TRVALY,PSC_TRVALY
0,25,8545430001,1975-09-21,M,2005-07-01,NaN,NaN,NaN,NaN,506877,91943
1,25,3362830001,1968-10-01,M,2005-07-01,NaN,NaN,NaN,NaN,508039,90067
2,25,5682830001,1988-10-16,M,2005-07-01,NaN,NaN,NaN,NaN,507938,90028
3,25,6245440001,1945-07-13,Z,2005-07-01,NaN,NaN,NaN,NaN,507865,90083
4,25,9161440001,1939-11-21,Z,2005-07-01,NaN,NaN,NaN,NaN,508098,90042


In [ ]:
insurees27.dtypes

KOD_ZP               int64
BIC_POI             object
DATUM_NAROD         object
POHLAVIE            object
DATUM_POI_OD        object
DATUM_POI_DO        object
KONIEC_POI_DOVOD    object
KOD_PRECHOD         object
PSC_PRECHOD         object
KOD_TRVALY          object
PSC_TRVALY          object
dtype: object

In [ ]:
insurees27.head()

,KOD_ZP,BIC_POI,DATUM_NAROD,POHLAVIE,DATUM_POI_OD,DATUM_POI_DO,KONIEC_POI_DOVOD,KOD_PRECHOD,PSC_PRECHOD,KOD_TRVALY,PSC_TRVALY
0,27,100000000714,2004-10-01,Z,2012-01-01,NaN,NaN,NaN,NaN,510173,02712
1,27,100000000740,1937-10-03,Z,2007-01-01,NaN,NaN,NaN,NaN,510106,02801
2,27,100000001674,1982-04-18,M,2022-01-01,NaN,NaN,560049,04945,526185,04935
3,27,100000001706,1979-10-01,M,2007-01-01,NaN,NaN,NaN,NaN,560031,04801
4,27,100000002644,1979-04-30,Z,2020-01-01,NaN,NaN,NaN,NaN,560022,04951


### Concatenate all insurees

In [ ]:
insurees_allhic = pd.concat([insurees24, insurees25, insurees27], ignore_index=True)

In [ ]:
insurees_allhic.shape

(5540524, 11)

In [ ]:
insurees_allhic['BIC_POI'].nunique()

5470836

In [ ]:
insurees_allhic.dtypes

KOD_ZP               int64
BIC_POI             object
DATUM_NAROD         object
POHLAVIE            object
DATUM_POI_OD        object
DATUM_POI_DO        object
KONIEC_POI_DOVOD    object
KOD_PRECHOD         object
PSC_PRECHOD         object
KOD_TRVALY          object
PSC_TRVALY          object
dtype: object

In [ ]:
insurees_allhic['KOD_TRVALY']

0          599816
1          522261
2          521299
3          522261
4          582514
            ...  
5540519       NaN
5540520       NaN
5540521       NaN
5540522       NaN
5540523       NaN
Name: KOD_TRVALY, Length: 5540524, dtype: object

## PZS GPS

In [ ]:
pzs_gps = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Jobs/IZA/Pacienti_obce/data/PZS_zakl_info_GPS_bezprepojeni.xlsx")

In [ ]:
pzs_gps['SIDZAR_KOD_OBEC'] = pzs_gps['SIDZAR_KOD_OBEC'].apply(lambda x: str(int(x)) if pd.notnull(x) else np.nan)

In [ ]:
pzs_gps['SIDZAR_KOD_OKRES_PZS'] = pzs_gps['SIDZAR_KOD_OKRES_PZS'].apply(lambda x: str(int(x)) if pd.notnull(x) else np.nan)

In [ ]:
pzs_gps.rename(columns={'IDENTIFZAR': 'kod_nem', 'SIDZAR_KOD_OBEC': 'KOD_OBEC_PZS', 'SIDZAR_KOD_OKRES_PZS': 'KOD_OKRES_PZS', 'lat_pzs_ifexists': 'lat_pzs', 'lng_pzs_ifexists': 'lng_pzs', 'NAZZAR': 'NAZOV_PZS'}, inplace=True)

In [ ]:
pzs_gps.head()

,kod_nem,PZS_short,NAZOV_PZS,KOD_OBEC_PZS,KOD_OKRES_PZS,lat_pzs,lng_pzs
0,P5848001,P58480,"Ministerstvo vnútra Slovenskej republiky, všeo...",501760,201,47.800111,17.664517
1,P5848002,P58480,"Ministerstvo vnútra Slovenskej republiky, všeo...",520004,702,48.933671,21.906581
2,P5848003,P58480,"Ministerstvo vnútra Slovenskej republiky, všeo...",508063,106,48.436960,17.018872
3,P5848004,P58480,"Ministerstvo vnútra Slovenskej republiky, všeo...",517402,511,49.219161,18.731891
4,P5848005,P58480,"Ministerstvo vnútra Slovenskej republiky, všeo...",528722,811,48.702044,21.655009


In [ ]:
pzs_gps.dtypes

kod_nem           object
PZS_short         object
NAZOV_PZS         object
KOD_OBEC_PZS      object
KOD_OKRES_PZS     object
lat_pzs          float64
lng_pzs          float64
dtype: object

## Patients residencies GPS

In [ ]:
pacient_gps = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Jobs/IZA/Pacienti_obce/data/obce_gps_simple.xlsx")

In [ ]:
pacient_gps.head()

,RESIDENCY,Obec_pacient_nazov,id_okres_pacient,Okres_pacient_nazov,okresne_mesto_zuj_pacient,okresne_mesto_pacient_nazov,lat_okres_pacient,lng_okres_pacient
0,501280,Nesvady,401,Komarno,501026,Komarno,47.765799,18.119551
1,504025,Sala,405,Sala,504025,Sala,48.156715,17.875971
2,504092,Trnovec nad Vahom,405,Sala,504025,Sala,48.156715,17.875971
3,555509,Zalesie,108,Senec,508217,Senec,48.219813,17.396061
4,504131,Velke Ulany,202,Galanta,503665,Galanta,48.189953,17.725027


In [ ]:
pacient_gps.dtypes

RESIDENCY                       object
Obec_pacient_nazov              object
id_okres_pacient                object
Okres_pacient_nazov             object
okresne_mesto_zuj_pacient       object
okresne_mesto_pacient_nazov     object
lat_okres_pacient              float64
lng_okres_pacient              float64
dtype: object

musim prehodit "RESIDENCY" na string, inak neskor nefunguje merge

In [ ]:

pacient_gps['RESIDENCY'] = pacient_gps['RESIDENCY'].apply(lambda x: str(int(x)) if pd.notnull(x) and str(x).isnumeric() else x)


# MS attribution

toto by mal byt engine, kde si hospitalizacie (jedna je na viacerych riadkoch) vytriedim, aby mi idealne na konci ostal jeden riadok s jednou relevantnou MS. Do budcna idealne aj s info o oddeleni a LOSe pacienta. Nateraz davam zjednodusene

In [ ]:
hospitalizations_sorted = hospitalizations.sort_values(by=['subor', 'prior'], ascending=[True, True])
hospitalizations_filtered = hospitalizations_sorted.drop_duplicates(subset='ID_HP_ZP', keep='first')

In [ ]:
hospitalizations_filtered.shape

(1005472, 21)

In [ ]:
hospitalizations_filtered.head()

,id_hosp_nem,vek,HMOTNOST,UPV,typ_zs,vykon_lab,vykon,kod_ms,ms_name,drg,icd,icd_vedlajsia,icd_any,joiner,subor,prior,tazkosti,sig_op,vek_skupina,kod_nem,ID_HP_ZP
3644223,U43_22304902_P4070703,0,870,0,DRG,8p107_20220425_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P4070703,U43_22304902
3644230,56771_22001266_P3881101,0,1350,744,DRG,8p133_20220313_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,56771_22001266
3644231,56771_22005282_P3881101,0,640,613,DRG,8p133_20220410_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,56771_22005282
3644234,39507_22001949_P8948301,0,4000,24,DRG,8p133_20220316_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P8948301,39507_22001949
3644236,39507_22002222_P8948301,0,1100,71,DRG,8p107_20220324_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P8948301,39507_22002222


In [ ]:
hospitalizations_filtered['ID_HP_ZP'].isna().sum()

0

In [ ]:
hospitalizations_filtered_idhpzpNaNs = hospitalizations_filtered[hospitalizations_filtered['ID_HP_ZP'].isin(['NaN', 'Nan', 'nan'])]

In [ ]:
hospitalizations_filtered_idhpzpNaNs.shape


(0, 21)

cize ozaj vsade ma ID_HP_ZP nejaku hodnotu

## Attempt 1 - not really successful

In [ ]:
## List of your dataframes (df1 - df6)
#dataframes = [uzs_jzs_24, hp_24, uzs_jzs_25, hp_25, uzs_jzs_27, hp_27]


## Initialize a new dataframe to store the merged data
#merged_data = hospitalizations_filtered

## Merge each of the dataframes with "hospitalizations_filtered" based on "ID_HP_ZP"
#for df in dataframes:
   # merged_data = merged_data.merge(df, on="ID_HP_ZP", how="left")

In [ ]:
#merged_data.head()

In [ ]:
#merged_data.shape

In [ ]:
#merged_data.dtypes

In [ ]:
## Initialize a copy of "hospitalizations_filtered" as the merged dataframe
#merged_data = hospitalizations_filtered.copy()

## Merge each of the dataframes with "merged_data" based on "ID_HP_ZP"
#for df in dataframes:
    #common_columns = set(merged_data.columns) & set(df.columns)
    #new_columns = set(df.columns) - set(merged_data.columns)

    ## Update existing columns
    #merged_data = pd.merge(merged_data, df, on="ID_HP_ZP", how="left")

    ## Add new columns
    #if new_columns:
       # merged_data = pd.concat([merged_data, df[['ID_HP_ZP']] + list(new_columns)], axis=1)
#
## If you need to fill NaN values with a specific value (e.g., 0), you can do so with the fillna method
#merged_data = merged_data.fillna(0)

# Merging all dataframes to get BIC_POI (Attempt 2)

## Concat all uzs_jzs and hp dataframes into one dataframe

Let's concat all dataframes one under each other into one dataframe

In [ ]:
dataframes = [uzs_jzs_allhic, hp_24, hp_25, hp_27] # znema od verzie 3.9, uz len uzs_jzs_allhic spracovane v druhom datasete

uzs_jzs_hp_allhic = pd.concat(dataframes, ignore_index=True)

In [ ]:
uzs_jzs_hp_allhic.shape

(1974961, 40)

In [ ]:
uzs_jzs_hp_allhic.dtypes

KOD_ZP               int64
BIC_POI            float64
ROK_MESIAC          object
PZS_12              object
ID_HP_ZP            object
NOVORODENEC         object
DATUM_OD            object
DATUM_DO            object
POHYB_POI           object
DGN_PRIJ            object
DGN_PREP            object
KOD_VYKON_JZS       object
KOD_VYKON_OPER      object
VYKON_TYP           object
HOSP_TYP            object
PZS_ODOSIELATEL     object
DATUM_INDIKACIE     object
STAROST_TYP         object
ID_HP_PZS           object
PZS_6               object
HP_DLZKA           float64
PRIEPUSTKA_DNI     float64
VEK_DNI            float64
VEK_ROKY           float64
HMOTNOST           float64
UPV_DLZKA          float64
DATUM_NAROD         object
PRIJ_DRUH          float64
PRIJ_DOVOD         float64
PREP_DOVOD         float64
HDG                 object
HDG_LOK             object
DRG_SKUP            object
DRG_ERV             object
ZLUCENE_HP          object
UHRADA_HP           object
UHRADA_PP           object
U

In [ ]:
uzs_jzs_hp_allhic.head()

,KOD_ZP,BIC_POI,ROK_MESIAC,PZS_12,ID_HP_ZP,NOVORODENEC,DATUM_OD,DATUM_DO,POHYB_POI,DGN_PRIJ,DGN_PREP,KOD_VYKON_JZS,KOD_VYKON_OPER,VYKON_TYP,HOSP_TYP,PZS_ODOSIELATEL,DATUM_INDIKACIE,STAROST_TYP,ID_HP_PZS,PZS_6,HP_DLZKA,PRIEPUSTKA_DNI,VEK_DNI,VEK_ROKY,HMOTNOST,UPV_DLZKA,DATUM_NAROD,PRIJ_DRUH,PRIJ_DOVOD,PREP_DOVOD,HDG,HDG_LOK,DRG_SKUP,DRG_ERV,ZLUCENE_HP,UHRADA_HP,UHRADA_PP,UHRADA_IMZS,UHRADA_EMZS,KONTRAKT
0,24,3920291.0,2022-03,P38811005101,56771_22089850,NaN,NaN,2022-03-04,I005,F104,F104,NaN,NaN,N,A,N77927005201,NaN,UH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24,4039170.0,2022-07,P89851069101,31064_22004566,NaN,2022-07-16,2022-07-22,I001,I2514,I2514,NaN,NaN,N,D,N33067278101,NaN,UH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,24,3201459.0,2022-09,N80847051101,30909_22003283,1NO,NaN,2022-09-01,I008,Z380,Z380,NaN,NaN,N,A,NaN,NaN,DRG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,24,7744470.0,2022-01,P76995229101,60666_22000229,NaN,2022-01-22,2022-01-27,I009,C541,C541,NaN,NaN,O,A,P76995025103,NaN,UH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,24,3267233.0,2022-02,P85363005101,59946_22000645,NaN,NaN,2022-02-10,I020,F078,F078,NaN,NaN,N,A,NaN,NaN,UH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


TO DO Do buducna -  z tohto by sa dal vycitat LOS, mozni aj oddelenie a ine uzitocne info. ALe pacienti su na viacerych riadkoch

In [ ]:
uzs_jzs_hp_allhic['ID_HP_ZP'].nunique()

1110768

v tomto musi byt pes zakopany, mam tu malo tych ID_HP_ZP, preto potom neviem prist k BIC_POI

In [ ]:
uzs_jzs_hp_allhic['BIC_POI'].isna().sum()

6

In [ ]:
uzs_jzs_hp_allhic['ID_HP_ZP'].isna().sum()

26646

In [ ]:
# tu si ich chcem zobrazit, co to ma byt v prvom rade zac
nan_id_hp_zp_rows = uzs_jzs_hp_allhic[uzs_jzs_hp_allhic['ID_HP_ZP'].isna()]

#### Zahada unique values

tuto cast dole pridavam 17.1.2024 po tom, ako mi nebolo jasne, preco sa mi zrazu navysi pocet unique values ked to prehodim na string. Pytal som sa chatu GPT3.5

In [ ]:
# Check for unique values and their counts
unique_values = uzs_jzs_hp_allhic['ID_HP_ZP'].value_counts()

# Check if any values have leading/trailing whitespaces
whitespace_values = unique_values[unique_values.index.str.strip().str.len() != len(unique_values.index)]

# Print unique values and values with leading/trailing whitespaces
print("Unique Values:")
print(unique_values)
print("\nValues with Whitespace:")
print(whitespace_values)

Unique Values:
20222100040056    14
20223300044135    14
20226100060552    13
20223300085767    13
20222100044189    13
                  ..
uzs929409          1
uzs929408          1
uzs929407          1
uzs929406          1
100000504969       1
Name: ID_HP_ZP, Length: 1110768, dtype: int64

Values with Whitespace:
20222100040056    14
20223300044135    14
20226100060552    13
20223300085767    13
20222100044189    13
                  ..
uzs929409          1
uzs929408          1
uzs929407          1
uzs929406          1
100000504969       1
Name: ID_HP_ZP, Length: 1110768, dtype: int64


In [ ]:
import re

def has_special_characters(input_string):
    # Define a regular expression pattern to match special characters
    pattern = r'[!@#$%^&*()_+{}\[\]:;<>,.?~\\]'

    # Use the re.search() function to find a match
    if re.search(pattern, input_string):
        return True
    else:
        return False

# Example usage:
input_string = "Hello123!"
if has_special_characters(input_string):
    print("The string contains special characters.")
else:
    print("The string does not contain special characters.")


The string contains special characters.


In [ ]:
# Initialize a count variable to 0
#special_character_count = 0

# Assuming uzs_jzs_hp_allhic is your DataFrame
#for value in uzs_jzs_hp_allhic['ID_HP_ZP']:
    # Check if the value contains special characters
    #if has_special_characters(value):
        #special_character_count += 1

#print(f"Number of values with special characters: {special_character_count}")


### Prehodit na string je dolezite!!!

In [ ]:
# prehadzujem stlpec co potrebujem na string
uzs_jzs_hp_allhic['ID_HP_ZP'] = uzs_jzs_hp_allhic['ID_HP_ZP'].astype(str)

Je tu vela NaN hodnot v spojenom uzs_jzs_hp_all. Nemaju pre mna vyznam, musim ich dropnut

In [ ]:
uzs_jzs_hp_allhic['ID_HP_ZP'].isna().sum()

0

Zaujimave, ze po prehodeni na string mi tie NaN hodnoty "zmizli", rovnako ako aj pocet unique hodnot je vyssi

In [ ]:
uzs_jzs_hp_allhic = uzs_jzs_hp_allhic.dropna(subset=['ID_HP_ZP'])

In [ ]:
uzs_jzs_hp_allhic['ID_HP_ZP'].nunique()

1110769

In [ ]:
#uzs_jzs_hp_allhic.to_csv("uzs_jzs_hp_allhic.csv")
# chcel som sa na tie data iba pozriet

## Merge uzs_jzs_hp_all with hospitalizations

### Merge

In [ ]:
merged_hospitalizations = hospitalizations_filtered.merge(uzs_jzs_hp_allhic[['ID_HP_ZP', 'BIC_POI']], on='ID_HP_ZP', how='left')

In [ ]:
# new
# Merge based on "ID_HP_ZP" with a custom condition using str.contains
#merged_hospitalizations = hospitalizations_filtered.merge(uzs_jzs_hp_allhic,
                                                           #left_on=uzs_jzs_hp_allhic['ID_HP_ZP'].str.replace('_', ''),
                                                           #right_on=hospitalizations_filtered['ID_HP_ZP'].str.replace('_', ''),
                                                           #how='left')

In [ ]:
merged_hospitalizations.shape

(1816929, 22)

In [ ]:
merged_hospitalizations.dtypes

id_hosp_nem       object
vek                int64
HMOTNOST           int64
UPV                int64
typ_zs            object
vykon_lab         object
vykon             object
kod_ms            object
ms_name           object
drg               object
icd               object
icd_vedlajsia     object
icd_any           object
joiner            object
subor              int64
prior              int64
tazkosti           int64
sig_op             int64
vek_skupina       object
kod_nem           object
ID_HP_ZP          object
BIC_POI          float64
dtype: object

In [ ]:
merged_hospitalizations['ID_HP_ZP'].nunique()

1005472

zaujimave, tych unique hodnot mam tolko, kolko "ma byt", nabuduce skusit porozmyslat preco sa mi tam nalepilo nieco 2x z tych ID HP ZP. Niekde tam musia byt dake NaN values

In [ ]:
merged_hospitalizations['BIC_POI'].isna().sum()

982

In [ ]:
merged_hospitalizations

,id_hosp_nem,vek,HMOTNOST,UPV,typ_zs,vykon_lab,vykon,kod_ms,ms_name,drg,icd,icd_vedlajsia,icd_any,joiner,subor,prior,tazkosti,sig_op,vek_skupina,kod_nem,ID_HP_ZP,BIC_POI
0,U43_22304902_P4070703,0,870,0,DRG,8p107_20220425_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P4070703,U43_22304902,7.363907e+06
1,U43_22304902_P4070703,0,870,0,DRG,8p107_20220425_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P4070703,U43_22304902,7.363907e+06
2,56771_22001266_P3881101,0,1350,744,DRG,8p133_20220313_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,56771_22001266,9.364222e+06
3,56771_22001266_P3881101,0,1350,744,DRG,8p133_20220313_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,56771_22001266,9.364222e+06
4,56771_22005282_P3881101,0,640,613,DRG,8p133_20220410_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,56771_22005282,8.704994e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1816924,20226100088895_P8109501,51,-1,-1,DRG,8n2362_20220713_Z,.*,35-12,Iné ochorenia NS,.*,m791,.*,.*,icd,19,62655,0,0,age19,P8109501,20226100088895,3.523812e+09
1816925,100000462863_N2200101,74,-1,-1,DRG,34011_20220728_Z,.*,35-12,Iné ochorenia NS,.*,m791,.*,.*,icd,19,62655,0,0,age19,N2200101,100000462863,1.000009e+11
1816926,100000462863_N2200101,74,-1,-1,DRG,34011_20220728_Z,.*,35-12,Iné ochorenia NS,.*,m791,.*,.*,icd,19,62655,0,0,age19,N2200101,100000462863,1.000009e+11
1816927,100000462863_N2200101,74,-1,-1,DRG,34011_20220728_Z,.*,35-12,Iné ochorenia NS,.*,m791,.*,.*,icd,19,62655,0,0,age19,N2200101,100000462863,1.000009e+11


### Skusanie s merged_hospitalizations

In [ ]:
#merged_hospitalizations.to_csv("merged_hospitalizations_20231025.csv")
# chcel som sa na tie data iba pozriet

In [ ]:
merged_hospitalizations_droppedNaN = merged_hospitalizations.dropna(subset=['BIC_POI'])

In [ ]:
merged_hospitalizations_droppedNaN['ID_HP_ZP'].nunique()

1004490

#### odbocka - uzsXXXX problem

In [ ]:
# Count the number of values in the "ID_HP_ZP" column that start with "uzs"
uzs_count = (merged_hospitalizations['ID_HP_ZP'].str.startswith('uzs')).sum()

print("Number of values starting with 'uzs' in 'ID_HP_ZP' column:", uzs_count)

Number of values starting with 'uzs' in 'ID_HP_ZP' column: 256278


Takze vacsina mojich NaNs v BIC_POI v spojenom dataframe su tie s ID_HP_ZP "uzs123456..." hodnoty, o ktorych Peto vravel, ze ich manualne doplnal. Tie sa nenachadzaju v ID_HP_ZP v uzs_jzs_allhic. Preto mi to asi nema cez co spojit. Hoci BIC_POI v uzs_jzs_allhic su takmer vsade

In [ ]:
subset_idhpzp_startingwith_uzs = merged_hospitalizations[merged_hospitalizations['ID_HP_ZP'].str.startswith('uzs')]

In [ ]:
subset_idhpzp_startingwith_uzs.shape

(256278, 22)

In [ ]:
list_uzs_peto = list(subset_idhpzp_startingwith_uzs['ID_HP_ZP'])

In [ ]:
# Save to a JSON file (can also use CSV or TXT based on your preference)
with open('/content/drive/MyDrive/Colab Notebooks/Jobs/IZA/Pacienti_obce/data/list_uzs_peto.json', 'w') as file:
    json.dump(list_uzs_peto, file)

In [ ]:
# Extract the numeric part from 'ID_HP_ZP' and convert to integer
subset_idhpzp_startingwith_uzs['numeric_part'] = subset_idhpzp_startingwith_uzs['ID_HP_ZP'].str[3:].astype(int)

# Find the minimum value in the numeric part
min_value = subset_idhpzp_startingwith_uzs['numeric_part'].min()

# Find the corresponding 'ID_HP_ZP' value
min_uzs_value = subset_idhpzp_startingwith_uzs[subset_idhpzp_startingwith_uzs['numeric_part'] == min_value]['ID_HP_ZP'].iloc[0]

print(min_uzs_value)

uzs314310


<ipython-input-94-0c566fbc4677>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_idhpzp_startingwith_uzs['numeric_part'] = subset_idhpzp_startingwith_uzs['ID_HP_ZP'].str[3:].astype(int)


***koniec odbocky

In [ ]:
unique_merged_hospitalizations = merged_hospitalizations.drop_duplicates(subset='ID_HP_ZP', keep='first')

In [ ]:
unique_merged_hospitalizations.shape

(1005472, 22)

In [ ]:
unique_merged_hospitalizations['BIC_POI'].isna().sum()

982

Nabuduce: problem budu joiny, ale asi by som nadalej skusal left a problem moze byt aj v tomto "uzs_jzs_hp_allhic['ID_HP_ZP'].nunique()", tych unique je len okolo 700daco tis., ale ten shape je inak obrovsky celeho suboru. Mozno tam mam vela duplicit a preto mi potom hadze NaN hodnoty na BIC_POI? K joinom este tu https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html

In [ ]:
# Remove underscores from "ID_HP_ZP" in both dataframes
#hospitalizations_filtered['ID_HP_ZP_cleaned'] = hospitalizations_filtered['ID_HP_ZP'].str.replace('_', '')
#uzs_jzs_hp_allhic['ID_HP_ZP_cleaned'] = uzs_jzs_hp_allhic['ID_HP_ZP'].str.replace('_', '')

# Merge based on the cleaned "ID_HP_ZP" columns
#merged_hospitalizations = hospitalizations_filtered.merge(uzs_jzs_hp_allhic, on='ID_HP_ZP_cleaned', how='left')

In [ ]:
merged_hospitalizations['BIC_POI'].isna().sum()

982

In [ ]:
# Find duplicate rows based on the 'ID_HP_ZP' column
duplicates = merged_hospitalizations[merged_hospitalizations.duplicated(subset='ID_HP_ZP', keep=False)]

In [ ]:
duplicates.head()

,id_hosp_nem,vek,HMOTNOST,UPV,typ_zs,vykon_lab,vykon,kod_ms,ms_name,drg,icd,icd_vedlajsia,icd_any,joiner,subor,prior,tazkosti,sig_op,vek_skupina,kod_nem,ID_HP_ZP,BIC_POI
0,U43_22304902_P4070703,0,870,0,DRG,8p107_20220425_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P4070703,U43_22304902,7363907.0
1,U43_22304902_P4070703,0,870,0,DRG,8p107_20220425_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P4070703,U43_22304902,7363907.0
2,56771_22001266_P3881101,0,1350,744,DRG,8p133_20220313_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,56771_22001266,9364222.0
3,56771_22001266_P3881101,0,1350,744,DRG,8p133_20220313_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,56771_22001266,9364222.0
4,56771_22005282_P3881101,0,640,613,DRG,8p133_20220410_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,56771_22005282,8704994.0


In [ ]:
# Tento kod strasne dlho bezi, potom ho vypnem
# Group the data by the 'ID_HP_ZP' column
#grouped = merged_hospitalizations.groupby('ID_HP_ZP')

# Find and display rows with differences within each group
#for _, group in grouped:
    #if group.drop('ID_HP_ZP', axis=1).nunique().gt(1).any():
        #print(f"Differences in 'ID_HP_ZP' group: {group['ID_HP_ZP'].iloc[0]}")
        #print(group)
        #print('\n')

ZAVER tejto casti (k 30.10.2023) - asi by som mal pokracovat s tym "unique_merged_hospitalizations" dataframom. Ma okolo miliona unikatnych ID_HP_ZP a k tomu okolo 740 tis. BIC_POI. Tie NaN hodnoty v BIC_POI pramenia z nahodne vymyslenych "uzs..." udajov pre ID_HP_ZP, ktore sa v "uzs_jzs_allhic" datasete nenachadzaju, cize ich nema cez co naparovat (cize museli byt v hospitalizations). V tejto suvislosti je divne, kolko NaN hodnot bolo v "uzs_jzs_allhic['ID_HP_ZP'] pred a po zmene na stringy.

Nabuduce by som mohol skusit uz k BIC_POI priradit bydlisko a s tymi ostatnymi NaN si nejak poradit, na urovni nemocnice tym pacientom nejak random priradit mozno podla zvysnych cca 75% pacientov...

### Output

musim adaptovat BIC_POI do rovnakeho formatu ako pri insurees, cize najskor na integer bez desatinneho miesta a potom na string

In [ ]:
unique_merged_hospitalizations['BIC_POI'] = unique_merged_hospitalizations['BIC_POI'].apply(convert_to_string)

<ipython-input-103-4296606896af>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_merged_hospitalizations['BIC_POI'] = unique_merged_hospitalizations['BIC_POI'].apply(convert_to_string)


In [ ]:
unique_merged_hospitalizations.head()

,id_hosp_nem,vek,HMOTNOST,UPV,typ_zs,vykon_lab,vykon,kod_ms,ms_name,drg,icd,icd_vedlajsia,icd_any,joiner,subor,prior,tazkosti,sig_op,vek_skupina,kod_nem,ID_HP_ZP,BIC_POI
0,U43_22304902_P4070703,0,870,0,DRG,8p107_20220425_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P4070703,U43_22304902,7363907
2,56771_22001266_P3881101,0,1350,744,DRG,8p133_20220313_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,56771_22001266,9364222
4,56771_22005282_P3881101,0,640,613,DRG,8p133_20220410_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,56771_22005282,8704994
6,39507_22001949_P8948301,0,4000,24,DRG,8p133_20220316_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P8948301,39507_22001949,5016945
8,39507_22002222_P8948301,0,1100,71,DRG,8p107_20220324_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P8948301,39507_22002222,2668229


In [ ]:
unique_merged_hospitalizations.dtypes

id_hosp_nem      object
vek               int64
HMOTNOST          int64
UPV               int64
typ_zs           object
vykon_lab        object
vykon            object
kod_ms           object
ms_name          object
drg              object
icd              object
icd_vedlajsia    object
icd_any          object
joiner           object
subor             int64
prior             int64
tazkosti          int64
sig_op            int64
vek_skupina      object
kod_nem          object
ID_HP_ZP         object
BIC_POI          object
dtype: object

ZAVER 31.10.2023: mam "unique_merged_hospitalizations" s BIC_POI ako stringom a insurees24, 25 a 27 taktiez s BIC_POI ako strings a kodmi trvaleho a prechodneho bydliska.

Nabuduce:
- spojit insurees do jedneho datasetu, pozriet unique hodnoty atd
- skusit potom prilepit k "unique_merged_hospitalizations" kod bydliska v zmysle Petovho navodu, najskor prechodny a ak nie je tak trvaly, alebo aj aj a do nejakeho tretieho stlpca ten "aktualny"

# Merge "unique_merged_hospitalizations" with residencies

In [ ]:
df_hospitalizations_residency = unique_merged_hospitalizations.merge(insurees_allhic[['BIC_POI', 'KOD_PRECHOD', 'KOD_TRVALY']], on='BIC_POI', how='left')

In [ ]:
df_hospitalizations_residency.shape

(1008388, 24)

In [ ]:
df_hospitalizations_residency.dtypes

id_hosp_nem      object
vek               int64
HMOTNOST          int64
UPV               int64
typ_zs           object
vykon_lab        object
vykon            object
kod_ms           object
ms_name          object
drg              object
icd              object
icd_vedlajsia    object
icd_any          object
joiner           object
subor             int64
prior             int64
tazkosti          int64
sig_op            int64
vek_skupina      object
kod_nem          object
ID_HP_ZP         object
BIC_POI          object
KOD_PRECHOD      object
KOD_TRVALY       object
dtype: object

In [ ]:
df_hospitalizations_residency.head()

,id_hosp_nem,vek,HMOTNOST,UPV,typ_zs,vykon_lab,vykon,kod_ms,ms_name,drg,icd,icd_vedlajsia,icd_any,joiner,subor,prior,tazkosti,sig_op,vek_skupina,kod_nem,ID_HP_ZP,BIC_POI,KOD_PRECHOD,KOD_TRVALY
0,U43_22304902_P4070703,0,870,0,DRG,8p107_20220425_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P4070703,U43_22304902,7363907,NaN,529460
1,56771_22001266_P3881101,0,1350,744,DRG,8p133_20220313_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,56771_22001266,9364222,NaN,510025
2,56771_22005282_P3881101,0,640,613,DRG,8p133_20220410_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,56771_22005282,8704994,NaN,506524
3,39507_22001949_P8948301,0,4000,24,DRG,8p133_20220316_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P8948301,39507_22001949,5016945,NaN,522279
4,39507_22002222_P8948301,0,1100,71,DRG,8p107_20220324_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P8948301,39507_22002222,2668229,NaN,522872


In [ ]:
df_hospitalizations_residency['KOD_TRVALY'].isna().sum()

12853

In [ ]:
filtered_df = df_hospitalizations_residency[df_hospitalizations_residency['KOD_PRECHOD'].isin(['NaN', 'Nan', 'nan'])]

In [ ]:
filtered_df.shape

(948602, 24)

In [ ]:
df_hospitalizations_residency['BIC_POI'].isna().sum()
# tu je asi preto 0, lebo ja som aj tie NaNs premenil na stringy

0

In [ ]:
NaNcount = (df_hospitalizations_residency['BIC_POI'] == 'NaN').sum()
NaNcount
# ako som si myslel vyssie, skutocne NaNs su teraz stringy. Necelych 10 tis. BIC_POI nemalo ani KOD_TRVALY (asi)

982

In [ ]:
df_hospitalizations_residency['RESIDENCY'] = df_hospitalizations_residency['KOD_PRECHOD']

In [ ]:
df_hospitalizations_residency['RESIDENCY'] = np.where(
    df_hospitalizations_residency['KOD_PRECHOD'].str.lower().isin(['nan', 'n/a', 'missing']),
    df_hospitalizations_residency['KOD_TRVALY'],
    df_hospitalizations_residency['KOD_PRECHOD']
)

In [ ]:
df_hospitalizations_residency.head()

,id_hosp_nem,vek,HMOTNOST,UPV,typ_zs,vykon_lab,vykon,kod_ms,ms_name,drg,icd,icd_vedlajsia,icd_any,joiner,subor,prior,tazkosti,sig_op,vek_skupina,kod_nem,ID_HP_ZP,BIC_POI,KOD_PRECHOD,KOD_TRVALY,RESIDENCY
0,U43_22304902_P4070703,0,870,0,DRG,8p107_20220425_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P4070703,U43_22304902,7363907,NaN,529460,529460
1,56771_22001266_P3881101,0,1350,744,DRG,8p133_20220313_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,56771_22001266,9364222,NaN,510025,510025
2,56771_22005282_P3881101,0,640,613,DRG,8p133_20220410_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,56771_22005282,8704994,NaN,506524,506524
3,39507_22001949_P8948301,0,4000,24,DRG,8p133_20220316_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P8948301,39507_22001949,5016945,NaN,522279,522279
4,39507_22002222_P8948301,0,1100,71,DRG,8p107_20220324_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P8948301,39507_22002222,2668229,NaN,522872,522872


In [ ]:
df_hospitalizations_residency.dtypes

id_hosp_nem      object
vek               int64
HMOTNOST          int64
UPV               int64
typ_zs           object
vykon_lab        object
vykon            object
kod_ms           object
ms_name          object
drg              object
icd              object
icd_vedlajsia    object
icd_any          object
joiner           object
subor             int64
prior             int64
tazkosti          int64
sig_op            int64
vek_skupina      object
kod_nem          object
ID_HP_ZP         object
BIC_POI          object
KOD_PRECHOD      object
KOD_TRVALY       object
RESIDENCY        object
dtype: object

In [ ]:
df_hospitalizations_residency['RESIDENCY'].isna().sum()

12853

In [ ]:
df_hospitalizations_residency.shape

(1008388, 25)

In [ ]:
filtered_df = df_hospitalizations_residency[~df_hospitalizations_residency['KOD_PRECHOD'].str.lower().isin(['nan', 'n/a', 'missing'])]

# Display values in the other columns of the filtered DataFrame
filtered_df

,id_hosp_nem,vek,HMOTNOST,UPV,typ_zs,vykon_lab,vykon,kod_ms,ms_name,drg,icd,icd_vedlajsia,icd_any,joiner,subor,prior,tazkosti,sig_op,vek_skupina,kod_nem,ID_HP_ZP,BIC_POI,KOD_PRECHOD,KOD_TRVALY,RESIDENCY
13,8873_22015652_P4305901,0,3000,358,DRG,8p133_20220923_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P4305901,8873_22015652,8732094,505846,508217,505846
19,8873_22021515_P4305901,0,1310,217,DRG,8p133_20221212_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P4305901,8873_22021515,2612911,509086,509086,509086
22,20223300007936_P3881101,0,1050,1080,DRG,8p133_20211215_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,20223300007936,788251001,509302,517402,509302
23,20223300122755_P3881101,0,1110,8372,DRG,8p107_20211227_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,20223300122755,7870032101,NaN,NaN,NaN
29,20223300099848_P3881101,0,480,1539,DRG,8p133_20221007_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,20223300099848,7952962101,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008303,31064_22003209_P8985101,80,-2,-2,UH,NaN,.*,25-18,Bolesť na hrudníku,.*,z450,.*,.*,icd,19,62629,0,0,age19,P8985101,31064_22003209,2380959,599824,598224,599824
1008323,31064_22005941_P8985101,75,-2,-2,UH,NaN,.*,25-18,Bolesť na hrudníku,.*,z450,.*,.*,icd,19,62629,0,0,age19,P8985101,31064_22005941,4936810,522279,522741,522279
1008361,100000434715_P7794101,28,-1,-1,DRG,5l59153_20220309_P,.*,29-11,Dialýza,.*,z490,.*,.*,icd,19,62633,0,0,age19,P7794101,100000434715,100000912596,501263,501263,501263
1008364,20232100003023_N4223101,59,-2,-2,DRG,NaN,.*,27-09,Stav po transplantácii pečene,.*,z944,.*,.*,icd,19,62652,0,0,age19,N4223101,20232100003023,5691062001,516643,508438,516643


# Pripajanie GPS PZS a dalsich info (nazov)

output, s ktorym sa pre tento ucel dalej pracuje:

In [ ]:
df_hospitalizations_residency.head(3)

,id_hosp_nem,vek,HMOTNOST,UPV,typ_zs,vykon_lab,vykon,kod_ms,ms_name,drg,icd,icd_vedlajsia,icd_any,joiner,subor,prior,tazkosti,sig_op,vek_skupina,kod_nem,ID_HP_ZP,BIC_POI,KOD_PRECHOD,KOD_TRVALY,RESIDENCY
0,U43_22304902_P4070703,0,870,0,DRG,8p107_20220425_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P4070703,U43_22304902,7363907,NaN,529460,529460
1,56771_22001266_P3881101,0,1350,744,DRG,8p133_20220313_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,56771_22001266,9364222,NaN,510025,510025
2,56771_22005282_P3881101,0,640,613,DRG,8p133_20220410_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,56771_22005282,8704994,NaN,506524,506524


Kontrola Na hodnot pred pridanim nazvu

In [ ]:
df_hospitalizations_residency['kod_nem'].isna().sum()

0

In [ ]:
df_hospitalizations_residency[df_hospitalizations_residency['kod_nem'].isin(['NaN', 'Nan', 'nan', 'na', 'Na'])].shape

(0, 25)

In [ ]:
df_hospitalizations_residency.columns

Index(['id_hosp_nem', 'vek', 'HMOTNOST', 'UPV', 'typ_zs', 'vykon_lab', 'vykon',
       'kod_ms', 'ms_name', 'drg', 'icd', 'icd_vedlajsia', 'icd_any', 'joiner',
       'subor', 'prior', 'tazkosti', 'sig_op', 'vek_skupina', 'kod_nem',
       'ID_HP_ZP', 'BIC_POI', 'KOD_PRECHOD', 'KOD_TRVALY', 'RESIDENCY'],
      dtype='object')

In [ ]:
df_hospitalizations_residency = df_hospitalizations_residency.merge(pzs_gps[['kod_nem','NAZOV_PZS', 'KOD_OKRES_PZS', 'lat_pzs', 'lng_pzs']], on='kod_nem', how='left')

Kontrola dataframu a Na hodnot po

In [ ]:
df_hospitalizations_residency.head(3)

,id_hosp_nem,vek,HMOTNOST,UPV,typ_zs,vykon_lab,vykon,kod_ms,ms_name,drg,icd,icd_vedlajsia,icd_any,joiner,subor,prior,tazkosti,sig_op,vek_skupina,kod_nem,ID_HP_ZP,BIC_POI,KOD_PRECHOD,KOD_TRVALY,RESIDENCY,NAZOV_PZS,KOD_OKRES_PZS,lat_pzs,lng_pzs
0,U43_22304902_P4070703,0,870,0,DRG,8p107_20220425_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P4070703,U43_22304902,7363907,NaN,529460,529460,Univerzitná nemocnica Bratislava - všeobecná n...,105,48.100436,17.118492
1,56771_22001266_P3881101,0,1350,744,DRG,8p133_20220313_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,56771_22001266,9364222,NaN,510025,510025,"Univerzitná nemocnica Martin, všeobecná nemocnica",506,49.060635,18.919695
2,56771_22005282_P3881101,0,640,613,DRG,8p133_20220410_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,56771_22005282,8704994,NaN,506524,506524,"Univerzitná nemocnica Martin, všeobecná nemocnica",506,49.060635,18.919695


In [ ]:
df_hospitalizations_residency['NAZOV_PZS'].isna().sum()

0

In [ ]:
df_hospitalizations_residency['lat_pzs'].isna().sum()

0

In [ ]:
df_hospitalizations_residency[df_hospitalizations_residency['NAZOV_PZS'].isin(['NaN', 'Nan', 'nan', 'na', 'Na', ' ', ''])].shape

(0, 29)

mal som  nejakych 21tis. pripadov, ked nemam k danej nemocnici info ohladne nazvu a GPS - to uz neplati, lebo som do datasetu pridal info ku vsetkym kodom

In [ ]:
hospitals_with_NaN_names_gps = df_hospitalizations_residency[df_hospitalizations_residency['NAZOV_PZS'].isna()]

In [ ]:
hospitals_with_NaN_names_gps

,id_hosp_nem,vek,HMOTNOST,UPV,typ_zs,vykon_lab,vykon,kod_ms,ms_name,drg,icd,icd_vedlajsia,icd_any,joiner,subor,prior,tazkosti,sig_op,vek_skupina,kod_nem,ID_HP_ZP,BIC_POI,KOD_PRECHOD,KOD_TRVALY,RESIDENCY,NAZOV_PZS,KOD_OKRES_PZS,lat_pzs,lng_pzs


Zda sa ze NaNs tu robi P8321602, nieco so SZ ZiarnH/BS, overit nabuduce a pridat do suboru, z ktoreho sa mi nacitavaju udaje k PZS

In [ ]:
# Get list of values from ColumnA where ColumnB has NaN values
list_of_values = df_hospitalizations_residency['kod_nem'][df_hospitalizations_residency['lat_pzs'].isna()].tolist()
unique_missing_pzs = df_hospitalizations_residency['kod_nem'][df_hospitalizations_residency['lat_pzs'].isna()].unique().tolist()

print(len(list_of_values))
print(df_hospitalizations_residency['kod_nem'][df_hospitalizations_residency['lat_pzs'].isna()].nunique())
print(unique_missing_pzs)

0
0
[]


Update 29.11.2023 - vyriesene, vsetkym kodom PZS bol priradeny nazov PZS a GPS koordinaty, v subore "PZS_zakl_info_GPS_bezprepojeni.xlxs" tam, kde som nevedel, mam random GPS/ obec pridelenu (myslim ze vsetko je na BA), ale to je len par pripadov. Rovnako, zostava par "problemovych" PZS ako napr. PentaHospitals SK, ktory maju adresu v Digital Parku ale vieme ze realne poskytuju ZS inde. Rovnako kvoli transformacii Nemocnice ZnH (zo SZ na Agel) a zluceniu s Ban. Stiav. tu existuje viacero kodov a vykazovanie tychto nemocnic bude mess (#todo potom mozno nejak zlucit?)

#Priprajanie GPS okresu pacientov a dalsich info

In [ ]:
df_hospitalizations_residency = df_hospitalizations_residency.merge(pacient_gps[['RESIDENCY','Obec_pacient_nazov', 'id_okres_pacient', 'lat_okres_pacient', 'lng_okres_pacient']], on='RESIDENCY', how='left')

In [ ]:
df_hospitalizations_residency.head()

,id_hosp_nem,vek,HMOTNOST,UPV,typ_zs,vykon_lab,vykon,kod_ms,ms_name,drg,icd,icd_vedlajsia,icd_any,joiner,subor,prior,tazkosti,sig_op,vek_skupina,kod_nem,ID_HP_ZP,BIC_POI,KOD_PRECHOD,KOD_TRVALY,RESIDENCY,NAZOV_PZS,KOD_OKRES_PZS,lat_pzs,lng_pzs,Obec_pacient_nazov,id_okres_pacient,lat_okres_pacient,lng_okres_pacient
0,U43_22304902_P4070703,0,870,0,DRG,8p107_20220425_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P4070703,U43_22304902,7363907,NaN,529460,529460,Univerzitná nemocnica Bratislava - všeobecná n...,105,48.100436,17.118492,Bratislava - Petrzalka,105,48.111710,17.100531
1,56771_22001266_P3881101,0,1350,744,DRG,8p133_20220313_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,56771_22001266,9364222,NaN,510025,510025,"Univerzitná nemocnica Martin, všeobecná nemocnica",506,49.060635,18.919695,Rabca,507,49.403873,19.482042
2,56771_22005282_P3881101,0,640,613,DRG,8p133_20220410_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,56771_22005282,8704994,NaN,506524,506524,"Univerzitná nemocnica Martin, všeobecná nemocnica",506,49.060635,18.919695,Stara Tura,304,48.759090,17.829386
3,39507_22001949_P8948301,0,4000,24,DRG,8p133_20220316_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P8948301,39507_22001949,5016945,NaN,522279,522279,"Detská fakultná nemocnica Košice, všeobecná ne...",803,48.721161,21.238323,Michalovce,807,48.754228,21.910748
4,39507_22002222_P8948301,0,1100,71,DRG,8p107_20220324_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P8948301,39507_22002222,2668229,NaN,522872,522872,"Detská fakultná nemocnica Košice, všeobecná ne...",803,48.721161,21.238323,Pavlovce nad Uhom,807,48.754228,21.910748


In [ ]:
df_hospitalizations_residency['id_okres_pacient'].isna().sum()

13101

opat, velmi vela NaN hodnot...

In [ ]:
patients_with_NaN_id_okres_pacienta = df_hospitalizations_residency[df_hospitalizations_residency['id_okres_pacient'].isna()]

In [ ]:
patients_with_NaN_id_okres_pacienta.shape

(13101, 33)

In [ ]:
patients_with_NaN_id_okres_pacienta.columns

Index(['id_hosp_nem', 'vek', 'HMOTNOST', 'UPV', 'typ_zs', 'vykon_lab', 'vykon',
       'kod_ms', 'ms_name', 'drg', 'icd', 'icd_vedlajsia', 'icd_any', 'joiner',
       'subor', 'prior', 'tazkosti', 'sig_op', 'vek_skupina', 'kod_nem',
       'ID_HP_ZP', 'BIC_POI', 'KOD_PRECHOD', 'KOD_TRVALY', 'RESIDENCY',
       'NAZOV_PZS', 'KOD_OKRES_PZS', 'lat_pzs', 'lng_pzs',
       'Obec_pacient_nazov', 'id_okres_pacient', 'lat_okres_pacient',
       'lng_okres_pacient'],
      dtype='object')

In [ ]:
patients_with_NaN_id_okres_pacienta

,id_hosp_nem,vek,HMOTNOST,UPV,typ_zs,vykon_lab,vykon,kod_ms,ms_name,drg,icd,icd_vedlajsia,icd_any,joiner,subor,prior,tazkosti,sig_op,vek_skupina,kod_nem,ID_HP_ZP,BIC_POI,KOD_PRECHOD,KOD_TRVALY,RESIDENCY,NAZOV_PZS,KOD_OKRES_PZS,lat_pzs,lng_pzs,Obec_pacient_nazov,id_okres_pacient,lat_okres_pacient,lng_okres_pacient
23,20223300122755_P3881101,0,1110,8372,DRG,8p107_20211227_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,20223300122755,7870032101,NaN,NaN,NaN,"Univerzitná nemocnica Martin, všeobecná nemocnica",506,49.060635,18.919695,NaN,NaN,NaN,NaN
29,20223300099848_P3881101,0,480,1539,DRG,8p133_20221007_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,20223300099848,7952962101,NaN,NaN,NaN,"Univerzitná nemocnica Martin, všeobecná nemocnica",506,49.060635,18.919695,NaN,NaN,NaN,NaN
31,20223300119777_P3881101,0,1700,177,DRG,8p107_20221022_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,20223300119777,6297113001,NaN,NaN,NaN,"Univerzitná nemocnica Martin, všeobecná nemocnica",506,49.060635,18.919695,NaN,NaN,NaN,NaN
35,20226100019210_P4305901,0,3590,378,DRG,8p107_20220125_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P4305901,20226100019210,6496437001,NaN,NaN,NaN,"Národný ústav detských chorôb, špecializovaná ...",103,48.167618,17.090078,NaN,NaN,NaN,NaN
53,20226100104855_P4070703,0,735,-1,DRG,8p107_20220603_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P4070703,20226100104855,6993942101,NaN,NaN,NaN,Univerzitná nemocnica Bratislava - všeobecná n...,105,48.100436,17.118492,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1018962,20223300006592_P3881101,42,-2,-2,DRG,NaN,.*,32-04,Raritné vírusové ochorenia,.*,u109,.*,.*,icd,19,62529,0,0,age19,P3881101,20223300006592,7710147001,NaN,NaN,NaN,"Univerzitná nemocnica Martin, všeobecná nemocnica",506,49.060635,18.919695,NaN,NaN,NaN,NaN
1019356,uzs919063_P3205101,34,-2,-2,JZS,12q201_202203,.*,41-03,Predpôrodná starostlivosť pre komplikácie grav...,.*,z362,.*,.*,icd,19,62602,0,0,age19,P3205101,uzs919063,701093001,NaN,NaN,NaN,"SAGAX, s.r.o., špecializovaná ambulancia, Duna...",201,47.992324,17.615617,NaN,NaN,NaN,NaN
1019444,20226100160684_P7794101,122,-1,-1,DRG,NaN,.*,41-17,Hospitalizácia po pôrode,.*,z390,.*,.*,icd,19,62617,0,0,age19,P7794101,20226100160684,3213619101,NaN,NaN,NaN,"Nemocnica AGEL Komárno s. r. o., všeobecná ne...",401,47.766326,18.127247,NaN,NaN,NaN,NaN
1019462,20224300079065_N3306701,30,-1,-1,DRG,NaN,.*,41-17,Hospitalizácia po pôrode,.*,z390,.*,.*,icd,19,62617,0,0,age19,N3306701,20224300079065,3828261001,NaN,NaN,NaN,Fakultná nemocnica s poliklinikou J. A. Reiman...,707,48.996321,21.252668,NaN,NaN,NaN,NaN


In [ ]:
df_hospitalizations_residency['RESIDENCY'].isna().sum()


12853

Pri "RESIDENCY" mam stale 274280 NaN hodnot ako som mal aj predtym (to je tych cca 27% NaNs co som mal aj doteraz a problem bude niekde inde - realne nedostupne data, zle spajanie, ... - musim prejst s Petom). Ale musim mat aj 324tis. minus 274 tis. pripadov, kde mam nejaky kod, ale moja databaza ho nepozna. DORIESIT nabuduce - ak sa bude dat, doplnit databazu, pripadne podla trvaleho pobytu (tam sa tie kody zdaju byt "normalne", pri prechodnom su niektore mozno PSC?)

update 12.12.2023: po tom, co som aktualizoval zoznam obci aj s chybajucimi kodmi*, tak mam uz len minimum "novych" chybajucich bydlisk pacientov. Gro je v zasade to, co uz bolo od zaciatku.

* *kedze tie udaje (kody ZUJ) som nevedel nikde najst, kazdemu cudnemu ZUJ som priradil okres podla toho, kde vyraznej casti pripadov bola poskytnuta zdrav. star. Na konci manualne pridane do suboru "obce_gps_simple.xlsx". Nie je to mozno uplne najspravnejsia metodika, lebo ak by pre nejake ZUJ napr. 50XXXX zo 120 osetrenych pacientov pripadalo 109 na okres Bratislava, este to neznamena, ze za tou ZS necestovali (a to chcem presne zistit). Ale je to vysoko pravdepodobne, ze ti pacienti su z BA a v tomto priklade tych 11 bratislavskych bolo osetrenych mimo BA

## Filtering "real" NaNs in "id_okres_pacient"

In [ ]:
filtered_df_realNaNs_patient = df_hospitalizations_residency[df_hospitalizations_residency['id_okres_pacient'].isna() & df_hospitalizations_residency['RESIDENCY'].notna()]


In [ ]:
filtered_df_realNaNs_patient.shape

(248, 33)

In [ ]:
filtered_df_realNaNs_patient.head()

,id_hosp_nem,vek,HMOTNOST,UPV,typ_zs,vykon_lab,vykon,kod_ms,ms_name,drg,icd,icd_vedlajsia,icd_any,joiner,subor,prior,tazkosti,sig_op,vek_skupina,kod_nem,ID_HP_ZP,BIC_POI,KOD_PRECHOD,KOD_TRVALY,RESIDENCY,NAZOV_PZS,KOD_OKRES_PZS,lat_pzs,lng_pzs,Obec_pacient_nazov,id_okres_pacient,lat_okres_pacient,lng_okres_pacient
6508,100000463579_P4070701,0,2360,-1,DRG,8n2023_20220730_Z,.*,42-03,"Novorodenec, hmotnosť pri prijatí 2000-2499 g ...",p66c,.*,.*,.*,drg,10,65,0,0,age0,P4070701,100000463579,100000939795,NaN,NaN,NaN,Univerzitná nemocnica Bratislava - všeobecná n...,102,48.156523,17.150507,NaN,NaN,NaN,NaN
8481,100000425543_P3881101,0,2130,-1,DRG,8n252_20220126_Z,.*,42-03,"Novorodenec, hmotnosť pri prijatí 2000-2499 g ...",p66d,.*,.*,.*,drg,10,66,0,0,age0,P3881101,100000425543,100001182832,NaN,NaN,NaN,"Univerzitná nemocnica Martin, všeobecná nemocnica",506,49.060635,18.919695,NaN,NaN,NaN,NaN
14785,100000470208_P5634601,0,2960,-1,DRG,8n2022_20220808_Z,.*,42-04,"Novorodenec, hmotnosť pri prijatí od 2500 g be...",p67c,.*,.*,.*,drg,10,67,0,0,age0,P5634601,100000470208,100001124224,NaN,NaN,NaN,"GPN s.r.o., špecializovaná nemocnica, Bratislava",101,48.146730,17.094805,NaN,NaN,NaN,NaN
15258,100000448009_P4070701,0,3990,-1,DRG,8n254_20220523_Z,.*,42-04,"Novorodenec, hmotnosť pri prijatí od 2500 g be...",p67c,.*,.*,.*,drg,10,67,0,0,age0,P4070701,100000448009,100000886834,NaN,NaN,NaN,Univerzitná nemocnica Bratislava - všeobecná n...,102,48.156523,17.150507,NaN,NaN,NaN,NaN
15440,100000447998_P4070701,0,3690,-1,DRG,8n254_20220525_Z,.*,42-04,"Novorodenec, hmotnosť pri prijatí od 2500 g be...",p67c,.*,.*,.*,drg,10,67,0,0,age0,P4070701,100000447998,100000829344,NaN,NaN,NaN,Univerzitná nemocnica Bratislava - všeobecná n...,102,48.156523,17.150507,NaN,NaN,NaN,NaN


In [ ]:
# skusim si to exportovat
filtered_df_realNaNs_patient.to_csv("filtered_df_realNaNs_patient.csv")

In [ ]:
missing_residency_list = filtered_df_realNaNs_patient['RESIDENCY'].tolist()


In [ ]:
unique_missing_residency_count = filtered_df_realNaNs_patient['RESIDENCY'].nunique()
print(unique_missing_residency_count)

100


In [ ]:
# Getting the count of each unique value in the "RESIDENCY" column
missing_residency_counts = filtered_df_realNaNs_patient['RESIDENCY'].value_counts()

# Converting the Series to a DataFrame
unique_missing_residency_df = missing_residency_counts.reset_index()

# Renaming the columns appropriately
unique_missing_residency_df.columns = ['RESIDENCY', 'COUNT']


dataframe si stiahnem a budem sa s nim hrat v exceli, prihodim potom hodnoty do povodneho excelu a dataframe v casti "Pacients_residencies_GPS"

In [ ]:
unique_missing_residency_df.to_excel("unique_missing_residency_df.xlsx")

Momentalne (1.12.2023) som sa zasekol na tom, ze neviem najst k tymto chybajucim kodom nic, ani na internete. Budem patrat dalej. Jedna sa o celkom dost pripadov, mnohe zacinaju 580XXX, ale tam nie je ziadna uzemna suvislost, 580XXX mozu byt obce z uplne inych krajov. Peto v navode spomina, ze mozno nie vsetko sa bude dat priradit. Vytvoril som si ale v Exceli PIVOT ze kde najcastesjsie boli ostetreni. Pri vacsine tych najviac zastupenych ZUJ kodov nejaka oblast (napr. BA) dominuje. Jedna moznost je to proste takto podoplnat - ale to bude data skreslovat.

# Creating map

Situacia (12.12.2023) je nasledovna. Mam dataset s nemocnicami a pacientami, okrem asi 274 tis. missing udajov k bydliskam zvysok mam. Idem skusit vytvorit mapu s pomocou chat GPT, kde si po okresoch zdruzim pacientov a uvidim, kam cestuju

In [ ]:
df_hospitalizations_residency.head()

,id_hosp_nem,vek,HMOTNOST,UPV,typ_zs,vykon_lab,vykon,kod_ms,ms_name,drg,icd,icd_vedlajsia,icd_any,joiner,subor,prior,tazkosti,sig_op,vek_skupina,kod_nem,ID_HP_ZP,BIC_POI,KOD_PRECHOD,KOD_TRVALY,RESIDENCY,NAZOV_PZS,KOD_OKRES_PZS,lat_pzs,lng_pzs,Obec_pacient_nazov,id_okres_pacient,lat_okres_pacient,lng_okres_pacient
0,U43_22304902_P4070703,0,870,0,DRG,8p107_20220425_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P4070703,U43_22304902,7363907,NaN,529460,529460,Univerzitná nemocnica Bratislava - všeobecná n...,105,48.100436,17.118492,Bratislava - Petrzalka,105,48.111710,17.100531
1,56771_22001266_P3881101,0,1350,744,DRG,8p133_20220313_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,56771_22001266,9364222,NaN,510025,510025,"Univerzitná nemocnica Martin, všeobecná nemocnica",506,49.060635,18.919695,Rabca,507,49.403873,19.482042
2,56771_22005282_P3881101,0,640,613,DRG,8p133_20220410_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P3881101,56771_22005282,8704994,NaN,506524,506524,"Univerzitná nemocnica Martin, všeobecná nemocnica",506,49.060635,18.919695,Stara Tura,304,48.759090,17.829386
3,39507_22001949_P8948301,0,4000,24,DRG,8p133_20220316_Z,8p133,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P8948301,39507_22001949,5016945,NaN,522279,522279,"Detská fakultná nemocnica Košice, všeobecná ne...",803,48.721161,21.238323,Michalovce,807,48.754228,21.910748
4,39507_22002222_P8948301,0,1100,71,DRG,8p107_20220324_Z,8p107,43-05,"Nekonvenčná UPV (vysokofrekvenčná, NO ventilácia)",p,.*,.*,.*,"drg, vykon",10,1,0,0,age0,P8948301,39507_22002222,2668229,NaN,522872,522872,"Detská fakultná nemocnica Košice, všeobecná ne...",803,48.721161,21.238323,Pavlovce nad Uhom,807,48.754228,21.910748


In [ ]:
df_hospitalizations_residency[df_hospitalizations_residency['id_okres_pacient'].str.lower().isin(['nan','n/a','missing', 'na',' ',''])].shape

(0, 33)

In [ ]:
df_hospitalizations_residency['id_okres_pacient'].isna().sum()

13101

In [ ]:
df_hospitalizations_residency.to_csv("df_hospitalizations_residency.csv")

In [ ]:
#df_hospitalizations_residency['kod_ms'] = df_hospitalizations_residency['kod_ms'].astype(str)

zmeny vo verzii 3.9 s vytvorenim "uzsX" kodov pre uzs_jzs datasety sa zda pomohli, este je otazka, ci je to co sa mi pripojilo spravne, ale po tychto poslednych upravach uz nemam 274tis NaN hodnot pre okres pacienta, ale iba 13-tis.

nechapem ale ze df_hospitalizations_residency.shape mi narastol z 1007824 na 1008388